In [26]:
import requests

In [27]:
standings_url = 'https://fbref.com/en/comps/9/Premier-League-Stats'

In [28]:
data = requests.get(standings_url)

In [29]:
from bs4 import BeautifulSoup

In [30]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [31]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [32]:
data = requests.get(team_urls[0])

In [33]:
import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

/var/folders/1s/v5yx4rxx4yl2vxprxkcr5y6w0000gp/T/ipykernel_50035/4209044294.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [34]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [35]:
data = requests.get(f"https://fbref.com{links[0]}")

In [36]:
shooting = pd.read_html(data.text, match="Shooting")[0]

/var/folders/1s/v5yx4rxx4yl2vxprxkcr5y6w0000gp/T/ipykernel_50035/2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [37]:
shooting.head()

For Manchester City                                                        \
                 Date   Time                 Comp                Round  Day   
0          2024-08-10  15:00  FA Community Shield  FA Community Shield  Sat   
1          2024-08-18  16:30       Premier League          Matchweek 1  Sun   
2          2024-08-24  15:00       Premier League          Matchweek 2  Sat   
3          2024-08-31  17:30       Premier League          Matchweek 3  Sat   
4                 NaN    NaN                  NaN                  NaN  NaN   

                                              ... Standard                \
  Venue Result     GF     GA        Opponent  ...     Dist   FK PK PKatt   
0  Home      D  1 (7)  1 (6)  Manchester Utd  ...      NaN  NaN  0     0   
1  Away      W      2      0         Chelsea  ...     19.1  0.0  0     0   
2  Home      W      4      1    Ipswich Town  ...     17.8  1.0  1     1   
3  Away      W      3      1        West Ham  ...     15.0  1.0  0     0   
4   NaN     --    NaN    NaN             NaN  ...     16.7  2.0  1     1   

  Expected                            Unnamed: 25_level_0  
        xG npxG npxG/Sh  G-xG np:G-xG        Match Report  
0      NaN  NaN     NaN   NaN     NaN        Match Report  
1      0.8  0.8    0.07   1.2     1.2        Match Report  
2      3.3  2.6    0.20   0.7     0.4        Match Report  
3      3.0  3.0    0.13   0.0     0.0        Match Report  
4      NaN  NaN    0.13  10.0     9.0                 NaN  

[5 rows x 26 columns]

In [38]:
shooting.columns = shooting.columns.droplevel()

In [39]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [40]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-10,15:00,FA Community Shield,FA Community Shield,Sat,Home,D,1 (7),1 (6),Manchester Utd,...,4-2-3-1,Jarred Gillett,Match Report,Manchester City won on penalty kicks following...,9,1,NaN,NaN,0,0
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,4-2-3-1,NaN,Match Report,NaN,11,5,19.1,0.0,0,0
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,5-4-1,NaN,Match Report,NaN,13,4,17.8,1.0,1,1
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,4-2-3-1,Michael Oliver,Match Report,NaN,23,8,15.0,1.0,0,0


In [41]:
years = list(range(2025, 2020, -1))
all_matches = []

In [42]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [43]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text, 'lxml')
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(15)
        

/var/folders/1s/v5yx4rxx4yl2vxprxkcr5y6w0000gp/T/ipykernel_50035/2085619569.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/1s/v5yx4rxx4yl2vxprxkcr5y6w0000gp/T/ipykernel_50035/2085619569.py:22: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/1s/v5yx4rxx4yl2vxprxkcr5y6w0000gp/T/ipykernel_50035/2085619569.py:17: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/1s/v5yx4rxx4yl2vxprxkcr5y6w0000gp/T/ipyke

In [44]:
len(all_matches)

100

In [45]:
match_df = pd.concat(all_matches)

In [46]:
match_df.columns = [c.lower() for c in match_df.columns]

In [47]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2024-08-18,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,Chelsea,...,Match Report,NaN,11.0,5.0,19.1,0.0,0,0,2025,Manchester City
2,2024-08-24,15:00,Premier League,Matchweek 2,Sat,Home,W,4,1,Ipswich Town,...,Match Report,NaN,13.0,4.0,17.8,1.0,1,1,2025,Manchester City
3,2024-08-31,17:30,Premier League,Matchweek 3,Sat,Away,W,3,1,West Ham,...,Match Report,NaN,23.0,8.0,15.0,1.0,0,0,2025,Manchester City
0,2024-08-17,15:00,Premier League,Matchweek 1,Sat,Away,W,3.0,0.0,Everton,...,Match Report,NaN,10.0,5.0,13.8,0.0,0,0,2025,Brighton and Hove Albion
1,2024-08-24,12:30,Premier League,Matchweek 2,Sat,Home,W,2.0,1.0,Manchester Utd,...,Match Report,NaN,14.0,4.0,14.2,1.0,0,0,2025,Brighton and Hove Albion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,Match Report,NaN,8.0,1.0,18.2,0.0,0,0,2021,Sheffield United
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,Match Report,NaN,7.0,0.0,13.4,1.0,0,0,2021,Sheffield United
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,Match Report,NaN,10.0,3.0,18.5,0.0,0,0,2021,Sheffield United
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,Match Report,NaN,11.0,1.0,18.3,1.0,0,0,2021,Sheffield United


In [48]:
match_df.to_csv("matches.csv")